### Install the _Kubeflow-metadata_ library (Load prereqs)
_**Note:** Make sure to have run:_

```bash
kubectl port-forward --namespace kubeflow $(kubectl get pod --namespace kubeflow --selector="component=grpc-server,kustomize.component=metadata" --output jsonpath='{.items[0].metadata.name}') 8080:8080
```

In [1]:
# To use the latest publish `kubeflow-metadata` library, you can run:
!pip install kubeflow-metadata --user
# Install other packages:
!pip install pandas --user
# Then restart the Notebook kernel.

In [1]:
import pandas
from kubeflow.metadata import metadata
from datetime import datetime
from uuid import uuid4
import numpy as np

### Load all test cases
- All 5 columns
- 7 Columns worth of data
- Partials:
  - Active Execution
  - 3 columns
  - 4 columns
- Multiples
  - Multi-Input
  - Multi-Output
  - Multi-Execution

In [3]:
cases = [
    "All 5 columns",
    "7 Columns worth of data",
    "Active Execution",
    "3 columns",
    "4 columns"
]

ws = [
    metadata.Workspace(
        store=metadata.Store(grpc_host="localhost", grpc_port=8080),
        name="test_case_{}".format(i),
        description=x,
        labels={"n1": "v1"}) for i, x in enumerate(cases)]

In [4]:
runs = [
    metadata.Run(
        workspace=w,
        name="run-" + datetime.utcnow().isoformat("T") ,
        description="a run in ws_{}".format(i),
    ) for i, w in enumerate(ws)
]

In [5]:
execs = [
    metadata.Execution(
        name = "execution-" + datetime.utcnow().isoformat("T") ,
        workspace=w,
        run=runs[i],
        description=cases[i],
    ) for i, w in enumerate(ws)]

In [6]:
print('Created executions:', list(map(lambda x: x.id, execs)))

Created executions: [14, 15, 16, 17, 18]


### Let's create fake data sources, that can be shared by our sources

In [7]:
get_date_set_version = lambda: "data_set_version_" + str(uuid4())
fileSources = [
    metadata.DataSet(
        description="Sample file set 1",
        name="table-dump",
        owner="ap@kubeflow.org",
        uri="file://datasets/dump1",
        version=get_date_set_version(),
        query="SELECT * FROM mytable"),
    metadata.DataSet(
        description="Sample file set 2",
        name="cloud-table",
        owner="ap@kubeflow.org",
        uri="gs://cloud/table.csv",
        version=get_date_set_version(),
        query="SELECT * FROM mytable"),
]

how_many_sources = np.random.choice(len(fileSources), len(execs))
data_sets = []

for i, src_count in enumerate(how_many_sources):
    exec = execs[i]
    ds = fileSources[0:src_count+1]
    ds = list(map(lambda x: exec.log_input(x), ds))
    print("Data sets:", ["{{id: {0.id}, version: '{0.version}'}}".format(d) for d in ds])
    data_sets.append(ds)

Data sets: ["{id: 19, version: 'data_set_version_33726879-8232-4a52-84a5-fe3bb110a63c'}", "{id: 20, version: 'data_set_version_f679ee55-55ef-4f66-a1e6-e9d59ea270fe'}"]
Data sets: ["{id: 19, version: 'data_set_version_33726879-8232-4a52-84a5-fe3bb110a63c'}", "{id: 20, version: 'data_set_version_f679ee55-55ef-4f66-a1e6-e9d59ea270fe'}"]
Data sets: ["{id: 19, version: 'data_set_version_33726879-8232-4a52-84a5-fe3bb110a63c'}", "{id: 20, version: 'data_set_version_f679ee55-55ef-4f66-a1e6-e9d59ea270fe'}"]
Data sets: ["{id: 19, version: 'data_set_version_33726879-8232-4a52-84a5-fe3bb110a63c'}", "{id: 20, version: 'data_set_version_f679ee55-55ef-4f66-a1e6-e9d59ea270fe'}"]
Data sets: ["{id: 19, version: 'data_set_version_33726879-8232-4a52-84a5-fe3bb110a63c'}"]


### Log a model

In [11]:
models = []
for i, exec in enumerate(execs):
    model_version = "model_version_{}".format(i)
    l = []
    l.append(exec.log_output(
        metadata.Model(
            name="MNIST",
            description="model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist",
            model_type="neural network",
            training_framework={
                "name": "tensorflow",
                "version": "v1.0"
            },
            hyperparameters={
                "learning_rate": 0.5,
                "layers": [10, 3, 1],
                "early_stop": True
            },
            version=model_version,
            labels={"mylabel": "l1"})))
    if np.random.choice(2, 1, p=[.6, .4]) == 1:
        l.append(exec.log_output(
            metadata.Model(
                name="SVHN",
                description="model to recognize house numbers on map images",
                owner="ap@kubeflow.org",
                uri="gcs://my-bucket/svhn",
                model_type="neural network",
                training_framework={
                    "name": "pytorch",
                    "version": "v1.0"
                },
                hyperparameters={
                    "learning_rate": 0.0001,
                    "layers": [10, 3, 1],
                    "early_stop": True
                },
                version=model_version,
                labels={"mylabel": "l1"})))
    print("Models:", ["{{id: {0.id}, version: '{0.version}'}}".format(d) for d in l])
    models.append(l)

Models: ["{id: 21, version: 'model_version_0'}"]
Models: ["{id: 23, version: 'model_version_1'}"]
Models: ["{id: 24, version: 'model_version_2'}"]
Models: ["{id: 25, version: 'model_version_3'}", "{id: 26, version: 'model_version_3'}"]
Models: ["{id: 27, version: 'model_version_4'}"]


### Log an evaluation of a model

In [25]:
for i, exec in enumerate(execs):
    for model in models[i]:
        for data_set in data_sets[i]:
            metrics = exec.log_output(
                metadata.Metrics(
                    name="{}-evaluation".format(model.name),
                    description="validating the {0.name} model to {0.description}".format(model),
                    owner=model.owner,
                    uri="gcs://my-bucket/{}-eval.csv".format(model.name.lower()),
                    data_set_id=str(data_set.id),
                    model_id=str(model.id),
                    metrics_type=metadata.Metrics.VALIDATION,
                    values={"accuracy": np.random.uniform(low=.6)},
                    labels={"mylabel": "l1"}))
print('Metrics created!')

Metrics created!


### Add Metadata for serving the model

In [27]:
for i, w in enumerate(ws):
    serving_application = metadata.Execution(
        name="serving model",
        workspace=w,
        description="an execution to represent model serving component",
    )
    for model in models[i]:
        # Noticed we use model name, version, uri to uniquely identify existing model.
        served_model = metadata.Model(
            name="MNIST",
            uri="gcs://my-bucket/mnist",
            version=model.version,
        )
        m=serving_application.log_input(served_model)
        print("Found the mode with id {0.id} and version '{0.version}'.".format(m))

Found the mode with id 21 and version 'model_version_0'.
Found the mode with id 23 and version 'model_version_1'.
Found the mode with id 24 and version 'model_version_2'.
Found the mode with id 25 and version 'model_version_3'.
Found the mode with id 25 and version 'model_version_3'.
Found the mode with id 27 and version 'model_version_4'.


### List all models in the workspace

In [29]:
pandas.DataFrame.from_dict(ws[1].list(metadata.Model.ARTIFACT_TYPE_NAME))

,id,workspace,run,version,owner,description,name,model_type,create_time,uri,training_framework,hyperparameters,labels,kwargs
0,23,test_case_1,run-2019-12-13T22:18:15.220690,model_version_1,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,2019-12-13T22:19:08.908693Z,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}


### Basic Lineage Tracking

In [30]:
# print("model id is %s\n" % model.id)
    
# model_events = ws1.store.get_events_by_artifact_ids([model.id])

# execution_ids = set(e.execution_id for e in model_events)
# print("All executions related to the model are {}".format(execution_ids))
# # assert execution_ids == set([serving_application.id, exec.id])

# trainer_events = ws1.store.get_events_by_execution_ids([exec.id])
# artifact_ids = set(e.artifact_id for e in trainer_events)
# print("All artifacts related to the training event are {}".format(artifact_ids))# assert artifact_ids == set([model.id, metrics.id, data_set.id])